In [ ]:
!git clone https://github.com/koreal6803/crypto_backtrader.git
%cd crypto_backtrader

Cloning into 'crypto_backtrader'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 13 (delta 0), reused 13 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.
/content/crypto_backtrader


In [ ]:
!pip install python-binance
!pip install bitmex
!pip install Backtesting
!pip install backtrader

     |████████████████████████████████| 368kB 19.8MB/s 
     |████████████████████████████████| 1.5MB 46.7MB/s 
     |████████████████████████████████| 2.6MB 36.7MB/s 
     |████████████████████████████████| 3.1MB 42.5MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 235kB 44.5MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 71kB 2.9MB/s 
     |████████████████████████████████| 143kB 17.0MB/s 
     |████████████████████████████████| 133kB 21.0MB/s 
  Created wheel for msgpack-python: filename=msgpack_python-0.5.6-cp36-cp36m-linux_x86_64.whl size=304236 sha256=b5f8aa1cb0c4bbc41f77f00207693fcd952797bf06ca8315de206806f8c08357
  Stored in directory: /root/.cache/pip/wheels/d5/de/86/7fa56fda12511be47ea0808f3502bc879df4e63ab168ec0406
  Created wheel for strict-rfc3339: filename=strict_rfc3339-0.7-cp36-none-any.whl size=18122 sha256=1cf3ca2

In [ ]:
from finlab import crypto

# geth the historical price
df = crypto.get_all_binance('BTCUSDT', '4h')

# calculate moving averages
sma1 = df.Close.rolling(20).mean()
sma2 = df.Close.rolling(60).mean()

# create long and short signal
signal_long = (sma1 > sma2) & (sma1.shift() < sma2.shift())
signal_short = (sma1 < sma2) & (sma1.shift() > sma2.shift())

# combine long and short signal
signal = signal_long.copy()
signal[signal_short] = -1

/usr/local/lib/python3.6/dist-packages/swagger_spec_validator/validator20.py:53: SwaggerValidationWarning: Found "$ref: #/definitions/UserPreferences" with siblings that will be overwritten. See https://stackoverflow.com/a/48114924 for more information. (path #/definitions/User/properties/preferences)
  ref_dict['$ref'], '/'.join(path),
/usr/local/lib/python3.6/dist-packages/backtesting/_plotting.py:44: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


2020-04-22 20:00:00 2020-08-31 05:00:18.861800
All caught up..!


In [ ]:
from backtesting import Backtest
from backtesting.lib import SignalStrategy
import pandas as pd

class Strategy(SignalStrategy):
    
    def init(self):
        super().init()
        
        # Precompute the two moving averages
        close = pd.Series(self.data.Close)
        sma1 = close.rolling(20).mean()
        sma2 = close.rolling(60).mean()
        
        # Precompute signal
        signal_long = (sma1 > sma2) & (sma1.shift() < sma2.shift())
        signal_short = (sma1 < sma2) & (sma1.shift() > sma2.shift())
        
        signal = signal_long
        signal[signal_short] = -1
        
        self.set_signal(signal)
        
        
    def next(self):
        super().next()

bt = Backtest(df, Strategy)
bt.run()

In [ ]:
bt.plot()